In [21]:
import pandas as pd

In [22]:
# ----------------------------------------------------------------------
# 1. Read data
# ----------------------------------------------------------------------
# Adjust the path/filename as needed:
dat = pd.read_csv('data/ProteinMatrix_sampleID_MapEC50_20240229.csv', index_col="Sample_ID")

/tmp/ipykernel_5829/561404019.py:5: DtypeWarning: Columns (0,5587,5588,5589,5590,5591,5592,5593,5596,5599,5600,5601,5603,5604,5606,5614) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv('data/ProteinMatrix_sampleID_MapEC50_20240229.csv', index_col="Sample_ID")


In [23]:

# ----------------------------------------------------------------------
# 2. Get protein Data
# ----------------------------------------------------------------------
p_idx_end = 5585
data_protein = dat.iloc[:, :p_idx_end].copy()

In [24]:

# ----------------------------------------------------------------------
# 3. Remove columns with only one unique value
# ----------------------------------------------------------------------
unique_counts = data_protein.nunique(dropna=False)
cols_to_remove = unique_counts[unique_counts == 1].index
data_protein.drop(columns=cols_to_remove, inplace=True)

# Keep only columns with "HUMAN" in their name (as in R code)
data_protein = data_protein.loc[:, data_protein.columns.str.contains("HUMAN")]



In [25]:
def na_imputation(series: pd.Series) -> pd.Series:
    """
    Replace NA values with 0.8 * min of the series (excluding NA).
    Equivalent to the na_imputation() function in R.
    """
    min_val = series.min(skipna=True)
    series_filled = series.fillna(min_val * 0.8)
    return series_filled


In [26]:

# ----------------------------------------------------------------------
# 4. Impute missing values
# ----------------------------------------------------------------------
data_protein = data_protein.apply(na_imputation)

In [27]:

# ----------------------------------------------------------------------
# 5. Log transform
# ----------------------------------------------------------------------
import numpy as np
data_protein = np.log(data_protein)


In [28]:

# ----------------------------------------------------------------------
# 6. Merge information of both sub xlsx of individual and combination drug information
# ----------------------------------------------------------------------
# This is necessary because the columns are split across two files in the original data. See description in bottom https://db.prottalks.com/views/PTV1.html
drugs = dat['pert_id'].astype(str)
# Convert to one-hot (pandas.get_dummies is similar to fastDummies in R)
data_drugs = pd.get_dummies(drugs, prefix='drug', dtype=float)


In [29]:

# Remove information on controls and combinations (which we deal with later)
for col in ['drug_no', 'drug_nan']:
    if col in data_drugs.columns:
        data_drugs.drop(columns=col, inplace=True)

# Multiply the entire drug matrix by 10 to indicate the dosage for single treatment experiments 10uM
data_drugs *= 10


In [30]:

# ----------------------------------------------------------------------
# 7. Identify combination rows (where drug_ was blank in R)
# ----------------------------------------------------------------------
combination_idx = dat.index[dat['pert_id'].isna()].tolist()  # all entries with nans correspond to drug combinations


In [31]:

# ----------------------------------------------------------------------
# 8. Adjust the one-hot encoding for combination and directly include doses
# ----------------------------------------------------------------------
for i in combination_idx:
    # 'drugIdAB' holds something like 'A B' for the two drugs
    drugAB = str(dat.loc[i, 'drugIdAB']).split()
    # Prepend 'drug_' to match the column naming
    drugAB_cols = [f"drug_{d}" for d in drugAB]
    
    anchor_dose = dat.loc[i, 'Anchor_dose']
    library_dose = dat.loc[i, 'Library_dose']
    
    # If the columns exist, set them accordingly - this skips the "drug_no" examples, so the controls for the combination exp
    if drugAB_cols[0] in data_drugs.columns:
        data_drugs.at[i, drugAB_cols[0]] = anchor_dose
    if drugAB_cols[1] in data_drugs.columns:
        data_drugs.at[i, drugAB_cols[1]] = library_dose

    # This way each row for drug combinations will have two non-zero values.


In [32]:

# ----------------------------------------------------------------------
# 9. Additional metadata columns
# ----------------------------------------------------------------------
data_additional = dat[['pert_time', 'protein_plate', 'machine', 'BioRep',
                        'Anchor_dose', 'Library_dose']].copy()
data_additional['Anchor_dose'] = data_additional['Anchor_dose'].fillna(0)
data_additional['Library_dose'] = data_additional['Library_dose'].fillna(0)


In [33]:

# ----------------------------------------------------------------------
# 10. Response data: singleDrug uses 'EC50', combos use 'Combo.IC50'
# ----------------------------------------------------------------------
data_response = dat['EC50'].copy()
combo_ic50 = dat['Combo.IC50'].copy()
data_response.loc[combination_idx] = combo_ic50.loc[combination_idx]


In [34]:

# ----------------------------------------------------------------------
# 11. Type and pertLabel columns
# ----------------------------------------------------------------------
# Mark as 'drugCombination' for combos, 'singleDrug' otherwise, 'noDrug' if 'no'
type_col = pd.Series(['singleDrug'] * len(dat), index=dat.index)
type_col.loc[combination_idx] = 'drugCombination'

combination_control_mask = dat["drugIdAB"] == 'no no'
type_col.loc[(drugs == 'no') | combination_control_mask] = 'noDrug' 

In [35]:

pertLabel = dat['pert_id'].astype(str).copy()
# For combos, set label to drugIdAB
pertLabel.loc[combination_idx] = dat.loc[combination_idx, 'drugIdAB'].astype(str)


In [36]:

# If 'no no' or 'no' means no drug, we can handle that similarly:
# data_response[pertLabel == 'no'] = np.inf  # e.g. replicate R's assignment of Inf

# ----------------------------------------------------------------------
# 12. Combine everything
# ----------------------------------------------------------------------
data_final = pd.concat([data_protein,
                        data_drugs,
                        data_additional], axis=1)
data_final['type'] = type_col.values
data_final['pertLabel'] = pertLabel.values
data_final['IC50'] = data_response.values
data_final['NY'] = dat['NY'].values  # If that column exists


In [37]:

# ----------------------------------------------------------------------
# 13. Log2 transform of singleDrug IC50 only
# ----------------------------------------------------------------------
single_drug_mask = (data_final['type'] == 'singleDrug') & data_final['IC50'].notna()
data_final.loc[single_drug_mask, 'IC50'] = np.log2(data_final.loc[single_drug_mask, 'IC50'])


In [38]:
data_final.head()

,Q8WUW1.Q8WUW1.BRK1_HUMAN.BRK1.Protein.BRICK1,Q9NRR4.Q9NRR4.RNC_HUMAN.DROSHA.Ribonuclease.3,Q99747.Q99747.SNAG_HUMAN.NAPG.Gamma.soluble.NSF.attachment.protein,P10398.P10398.ARAF_HUMAN.ARAF.Serine.threonine.protein.kinase.A.Raf,P05198.P05198.IF2A_HUMAN.EIF2S1.Eukaryotic.translation.initiation.factor.2.subunit.1,Q6P1N0.Q6P1N0.C2D1A_HUMAN.CC2D1A.Coiled.coil.and.C2.domain.containing.protein.1A,Q14807.Q14807.KIF22_HUMAN.KIF22.Kinesin.like.protein.KIF22,Q9H269.Q9H269.VPS16_HUMAN.VPS16.Vacuolar.protein.sorting.associated.protein.16.homolog,O00461.O00461.GOLI4_HUMAN.GOLIM4.Golgi.integral.membrane.protein.4,P49257.P49257.LMAN1_HUMAN.LMAN1.Protein.ERGIC.53,...,pert_time,protein_plate,machine,BioRep,Anchor_dose,Library_dose,type,pertLabel,IC50,NY
Sample_ID,,,,,,,,,,,,,,,,,,,,,
B1422,2.237325,4.951632,4.006710,4.239126,8.437013,3.734398,6.267629,3.58186,3.004411,8.912763,...,48,DU4475,D,1.0,0.0,0.0,singleDrug,#58,3.453649,Y
B5718,2.237325,4.951632,7.605178,4.239126,8.655340,3.734398,2.965665,3.58186,3.004411,9.082839,...,24,HCC1187,WAF,3.0,0.0,0.0,singleDrug,#25,6.409590,N
B3832,2.237325,4.951632,6.776822,4.239126,8.686840,3.734398,2.965665,3.58186,3.004411,8.705278,...,48,DU4475,D,2.0,0.0,0.0,singleDrug,#58,3.453649,Y
B3637,2.237325,4.951632,5.743561,4.239126,8.561968,3.734398,2.965665,3.58186,3.004411,9.072568,...,48,DU4475,D,3.0,0.0,0.0,singleDrug,#58,3.453649,Y
B532,2.237325,4.951632,4.006710,4.239126,8.487018,3.734398,2.965665,3.58186,3.004411,9.349450,...,24,DU4475,WAE,1.0,0.0,0.0,singleDrug,#58,3.453649,Y


In [39]:
data_final[data_final["type"] == "drugCombination"]["pertLabel"].value_counts()

pertLabel
#64 #56    200
#53 #64    172
#20 #64    161
#56 #53    156
#53 #32    151
#32 #47    149
#64 #58    141
#20 #32    135
#64 #32    134
#56 #58    107
#20 #53    106
#33 #21    101
#47 #64    100
#32 #58     91
#69 #20     89
#47 #56     87
#33 #56     82
#33 #37     81
#69 #64     80
#56 #32     79
#56 #21     78
#69 #21     78
#53 #21     77
#33 #32     75
#33 #64     74
#69 #53     73
#47 #21     73
#47 #37     72
#47 #53     72
#47 #20     70
#47 #8      69
#33 #20     64
#53 #37     63
#69 #37     61
#56 #37     61
#69 #58     61
#64 #8      59
#32 #8      56
#20 #56     56
#20 #8      55
#20 #21     54
#69 #47     54
#53 #8      53
#64 #21     52
#33 #47     52
#69 #8      51
#32 #37     47
#33 #53     46
#47 #58     45
#69 #56     42
#33 #8      42
#20 #37     41
#33 #58     38
#64 #37     38
#69 #32     38
#32 #21     36
#53 #58     34
#56 #8      34
#20 #58     24
Name: count, dtype: int64

In [40]:
data_final[data_final["type"] == "drugCombination"].loc[:, ["type", "pertLabel", "protein_plate", "machine", "BioRep", "Anchor_dose", "Library_dose", "pert_time"]].iloc[1000:2000,:]

,type,pertLabel,protein_plate,machine,BioRep,Anchor_dose,Library_dose,pert_time
Sample_ID,,,,,,,,
D2913,drugCombination,#32 #47,HCC1806,D,6.0,7.5,10.0,6
D3096,drugCombination,#33 #32,HCC1806,D,1.0,2.0,10.0,48
D2955,drugCombination,#33 #32,HCC1806,D,2.0,2.0,10.0,48
D2957,drugCombination,#33 #32,HCC1806,D,3.0,0.5,10.0,48
D2816,drugCombination,#33 #32,HCC1806,D,4.0,0.5,10.0,48
...,...,...,...,...,...,...,...,...
D4474,drugCombination,#33 #8,HCC1395,WAF,1.0,0.5,10.0,24
D4475,drugCombination,#33 #8,HCC1395,WAF,2.0,2.0,10.0,24
D4673,drugCombination,#33 #8,HCC1395,D,3.0,2.0,10.0,24


In [41]:
data_final[data_final["pertLabel"] == "#64 #56"][["Anchor_dose","Library_dose"]].value_counts()

Anchor_dose  Library_dose
0.10         4.0             61
4.00         10.0            53
0.01         4.0             43
1.00         10.0            43
Name: count, dtype: int64

In [42]:
data_final[data_final["type"] == "noDrug"]

,Q8WUW1.Q8WUW1.BRK1_HUMAN.BRK1.Protein.BRICK1,Q9NRR4.Q9NRR4.RNC_HUMAN.DROSHA.Ribonuclease.3,Q99747.Q99747.SNAG_HUMAN.NAPG.Gamma.soluble.NSF.attachment.protein,P10398.P10398.ARAF_HUMAN.ARAF.Serine.threonine.protein.kinase.A.Raf,P05198.P05198.IF2A_HUMAN.EIF2S1.Eukaryotic.translation.initiation.factor.2.subunit.1,Q6P1N0.Q6P1N0.C2D1A_HUMAN.CC2D1A.Coiled.coil.and.C2.domain.containing.protein.1A,Q14807.Q14807.KIF22_HUMAN.KIF22.Kinesin.like.protein.KIF22,Q9H269.Q9H269.VPS16_HUMAN.VPS16.Vacuolar.protein.sorting.associated.protein.16.homolog,O00461.O00461.GOLI4_HUMAN.GOLIM4.Golgi.integral.membrane.protein.4,P49257.P49257.LMAN1_HUMAN.LMAN1.Protein.ERGIC.53,...,pert_time,protein_plate,machine,BioRep,Anchor_dose,Library_dose,type,pertLabel,IC50,NY
Sample_ID,,,,,,,,,,,,,,,,,,,,,
B3658,2.237325,4.951632,4.006710,4.239126,8.653434,3.734398,2.965665,3.581860,3.004411,8.566749,...,0,DU4475,D,1.0,0.0,0.0,noDrug,no,NaN,N
B1448,2.237325,4.951632,4.006710,4.239126,8.453550,3.734398,2.965665,3.581860,3.004411,9.294939,...,0,DU4475,D,2.0,0.0,0.0,noDrug,no,NaN,N
B3663,2.237325,4.951632,4.006710,4.239126,8.542982,3.734398,2.965665,3.581860,6.656399,8.891363,...,0,DU4475,D,3.0,0.0,0.0,noDrug,no,NaN,N
B3848,2.237325,4.951632,6.817116,4.239126,8.832653,3.734398,2.965665,3.581860,3.004411,8.587171,...,0,DU4475,D,4.0,0.0,0.0,noDrug,no,NaN,N
B1450,5.775294,4.951632,7.222617,4.239126,9.390359,7.252529,2.965665,6.480123,3.004411,8.755834,...,0,DU4475,WAF,5.0,0.0,0.0,noDrug,no,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,6.649176,4.951632,8.165378,6.984698,9.537657,7.887460,2.965665,3.581860,3.004411,10.376580,...,0,HCC38,WAE,1.0,0.0,0.0,noDrug,no,NaN,N
2478,6.945590,4.951632,8.193328,6.970505,9.623952,6.878545,7.349964,6.310449,3.004411,10.159683,...,0,HCC38,WAE,2.0,0.0,0.0,noDrug,no,NaN,N
5550,8.260682,9.808451,7.964323,4.239126,9.640687,7.306545,2.965665,3.581860,3.004411,10.042253,...,0,HCC38,WAE,3.0,0.0,0.0,noDrug,no,NaN,N


In [43]:

# ----------------------------------------------------------------------
# 14. Save final data
# ----------------------------------------------------------------------
data_final.to_csv('data/preprocessed.csv')

print("Data preprocessing complete. Example final shape:", data_final.shape)

Data preprocessing complete. Example final shape: (15002, 5592)


In [50]:
data_final[(data_final["type"] == "drugCombination") & (data_final["pertLabel"] == "#64 #56")][["Anchor_dose", "Library_dose"]].value_counts()

Anchor_dose  Library_dose
0.10         4.0             61
4.00         10.0            53
0.01         4.0             43
1.00         10.0            43
Name: count, dtype: int64